<a href="https://colab.research.google.com/github/Minal2179/Practice/blob/master/Colab's_New_Code_Editor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
import pandas as pd
dsm_dataset_trace = pd.read_csv("Minal_DSM_411_643_800/DSM_411_643_800_trace_data.csv")
dsm_dataset_context = pd.read_csv("Minal_DSM_411_643_800/DSM_411_643_800_run_context.csv")
dsm_dataset_rate = pd.read_csv("Minal_DSM_411_643_800/DSM_411_643_800_sample_rate.csv")
dsm_dataset_step_data = pd.read_csv("Minal_DSM_411_643_800/DSM_411_643_800_stp_mapping.csv")
dsm_dataset_golden_runs = pd.read_csv("Minal_DSM_411_643_800/DSM_411_643_800_golden_runs.csv")
dsm_dataset_trace.head()

,Run,ARBTM_Stick_Flow,ARCLN_Stick_Flow,ARSH_Stick_Flow,ARTOP_Stick_Flow,Chamber_pm_wafer_count,HECLN_Stick_Flow,HX1_Resistivity,HX1_Temperature,HX4_Resistivity,HX4_Temperature,Heater_distance_from_shower_head,Heater_lift_position_from_Home,Heater_output_power_percent,NF3_Stick_Flow,O3_Stick_Flow,Pressure_Chamber,Pressure_Foreline,Purge03_Stick_pressure,RPS_Status,Step_Number,Temperature_Heater,Throttle_Valve_Position,UVLampGen_rActualPower_Bulb_A,UVLampGen_rActualPower_Bulb_B,UVLampRotator_rCurPosDegrees,UV_Lamp_On_Time,UV_Lamp_Stand_By_On_Time,Step_Name,stpid,CalcStepLoopCnt
0,422643.0,-5.7,-0.4,0.0,0.0,16207.0,0.0,2.300366e+06,74.945055,0.0,40.0,2.252,0.075,59.1,0.0,10032.0,0.018,0.009,0.0,0.0,1.0,299.7,90.0,0.0,0.0,0.0,1021.145165,12504.130297,Ar SH on,1,1.0
1,422643.0,-5.7,-0.4,0.0,0.0,16207.0,0.0,2.300366e+06,74.945055,0.0,40.0,2.252,0.075,59.1,0.0,10032.0,0.018,0.009,0.0,0.0,1.0,299.7,90.0,0.0,0.0,0.0,1021.145165,12504.130297,Ar SH on,1,1.0
2,422643.0,-5.7,-0.4,0.0,0.0,16207.0,0.0,2.300366e+06,74.945055,0.0,40.0,2.252,0.075,59.1,0.0,10032.0,0.018,0.009,0.0,0.0,1.0,299.7,90.0,0.0,0.0,0.0,1021.145165,12504.130297,Ar SH on,1,1.0
3,422643.0,-6.1,-0.6,1145.9,0.0,16207.0,0.0,2.300366e+06,74.945055,0.0,40.0,2.252,0.075,59.1,0.0,10032.0,0.018,0.009,0.0,0.0,1.0,299.7,90.0,0.0,0.0,0.0,1021.145165,12504.130297,Ar SH on,1,1.0
4,422643.0,-6.1,-0.6,1145.9,0.0,16207.0,0.0,2.300366e+06,74.945055,0.0,40.0,2.252,0.075,59.1,0.0,10032.0,0.018,0.009,0.0,0.0,1.0,299.7,90.0,0.0,0.0,0.0,1021.145165,12504.130297,Ar SH on,1,1.0


In [12]:
print('Initial dataset', dsm_dataset_trace.shape)
sample_rate= 0.201905
def perform_data_classification(dsm_dataset_trace, dsm_dataset_golden_runs):
    trace_data = dsm_dataset_trace.set_index(keys='Run')
    condition = trace_data.index.isin(dsm_dataset_golden_runs['Golden_Runs'].values)
    trace_data.index.name = 'id'
    test_data = trace_data.loc[~condition, :]
    train_data = trace_data.loc[condition, :]
    print("Test Data", test_data.shape)
    print("Train Data", train_data.shape)
    return train_data, test_data

train_data, test_data = perform_data_classification(dsm_dataset_trace, dsm_dataset_golden_runs)

Initial dataset (585151, 31)
Test Data (328240, 30)
Train Data (256911, 30)


In [13]:
def perform_data_preprocess(test_data, train_data):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    train_data = train_data.select_dtypes(include=numerics).dropna(axis=1, how='all')
    
    temp_data = test_data.loc[:, train_data.columns.values]
    combined_data = pd.concat([train_data, temp_data])
    
    #exclude records above 10,000
    sreduced_train_data = train_data.loc[:, combined_data.std() > 1e-4]
    sreduced_test_data = temp_data.loc[:, combined_data.std() > 1e-4]
    
    #normalize the above data
    train_norm_data = (sreduced_train_data - sreduced_train_data.min()) / (sreduced_train_data.max() - sreduced_train_data.min())
    train_norm_data.dropna(axis=1, how='any', inplace=True)
    test_norm_data = ((sreduced_test_data - sreduced_train_data.min()) / (sreduced_train_data.max() - sreduced_train_data.min())).loc[:, train_norm_data.columns.values] #?? why normalize test data differently then train data
    return test_norm_data, train_norm_data

test_norm_data, train_norm_data = perform_data_preprocess(test_data, train_data)

print("Test normalized data", test_norm_data.shape)
print("Train normalized data", train_norm_data.shape)

Test normalized data (328240, 21)
Train normalized data (256911, 21)


In [14]:
train_data.head()

,ARBTM_Stick_Flow,ARCLN_Stick_Flow,ARSH_Stick_Flow,ARTOP_Stick_Flow,Chamber_pm_wafer_count,HECLN_Stick_Flow,HX1_Resistivity,HX1_Temperature,HX4_Resistivity,HX4_Temperature,Heater_distance_from_shower_head,Heater_lift_position_from_Home,Heater_output_power_percent,NF3_Stick_Flow,O3_Stick_Flow,Pressure_Chamber,Pressure_Foreline,Purge03_Stick_pressure,RPS_Status,Step_Number,Temperature_Heater,Throttle_Valve_Position,UVLampGen_rActualPower_Bulb_A,UVLampGen_rActualPower_Bulb_B,UVLampRotator_rCurPosDegrees,UV_Lamp_On_Time,UV_Lamp_Stand_By_On_Time,Step_Name,stpid,CalcStepLoopCnt
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
422643.0,-5.7,-0.4,0.0,0.0,16207.0,0.0,2.300366e+06,74.945055,0.0,40.0,2.252,0.075,59.1,0.0,10032.0,0.018,0.009,0.0,0.0,1.0,299.7,90.0,0.0,0.0,0.0,1021.145165,12504.130297,Ar SH on,1,1.0
422643.0,-5.7,-0.4,0.0,0.0,16207.0,0.0,2.300366e+06,74.945055,0.0,40.0,2.252,0.075,59.1,0.0,10032.0,0.018,0.009,0.0,0.0,1.0,299.7,90.0,0.0,0.0,0.0,1021.145165,12504.130297,Ar SH on,1,1.0
422643.0,-5.7,-0.4,0.0,0.0,16207.0,0.0,2.300366e+06,74.945055,0.0,40.0,2.252,0.075,59.1,0.0,10032.0,0.018,0.009,0.0,0.0,1.0,299.7,90.0,0.0,0.0,0.0,1021.145165,12504.130297,Ar SH on,1,1.0
422643.0,-6.1,-0.6,1145.9,0.0,16207.0,0.0,2.300366e+06,74.945055,0.0,40.0,2.252,0.075,59.1,0.0,10032.0,0.018,0.009,0.0,0.0,1.0,299.7,90.0,0.0,0.0,0.0,1021.145165,12504.130297,Ar SH on,1,1.0
422643.0,-6.1,-0.6,1145.9,0.0,16207.0,0.0,2.300366e+06,74.945055,0.0,40.0,2.252,0.075,59.1,0.0,10032.0,0.018,0.009,0.0,0.0,1.0,299.7,90.0,0.0,0.0,0.0,1021.145165,12504.130297,Ar SH on,1,1.0


In [15]:
train_norm_data.head()

,ARBTM_Stick_Flow,ARCLN_Stick_Flow,ARSH_Stick_Flow,ARTOP_Stick_Flow,Chamber_pm_wafer_count,HX1_Resistivity,HX1_Temperature,HX4_Temperature,Heater_distance_from_shower_head,Heater_lift_position_from_Home,Heater_output_power_percent,O3_Stick_Flow,Pressure_Chamber,Pressure_Foreline,Step_Number,Temperature_Heater,Throttle_Valve_Position,UV_Lamp_On_Time,UV_Lamp_Stand_By_On_Time,stpid,CalcStepLoopCnt
id,,,,,,,,,,,,,,,,,,,,,
422643.0,0.088889,0.750,0.000000,0.000181,0.0,1.0,0.333333,0.5,1.0,0.0,0.181818,0.5,0.00006,0.004371,0.111111,0.3,1.0,0.0,0.0,0.0,0.0
422643.0,0.088889,0.750,0.000000,0.000181,0.0,1.0,0.333333,0.5,1.0,0.0,0.181818,0.5,0.00006,0.004371,0.111111,0.3,1.0,0.0,0.0,0.0,0.0
422643.0,0.088889,0.750,0.000000,0.000181,0.0,1.0,0.333333,0.5,1.0,0.0,0.181818,0.5,0.00006,0.004371,0.111111,0.3,1.0,0.0,0.0,0.0,0.0
422643.0,0.000000,0.625,0.635658,0.000181,0.0,1.0,0.333333,0.5,1.0,0.0,0.181818,0.5,0.00006,0.004371,0.111111,0.3,1.0,0.0,0.0,0.0,0.0
422643.0,0.000000,0.625,0.635658,0.000181,0.0,1.0,0.333333,0.5,1.0,0.0,0.181818,0.5,0.00006,0.004371,0.111111,0.3,1.0,0.0,0.0,0.0,0.0


In [16]:
test_data.head()

,ARBTM_Stick_Flow,ARCLN_Stick_Flow,ARSH_Stick_Flow,ARTOP_Stick_Flow,Chamber_pm_wafer_count,HECLN_Stick_Flow,HX1_Resistivity,HX1_Temperature,HX4_Resistivity,HX4_Temperature,Heater_distance_from_shower_head,Heater_lift_position_from_Home,Heater_output_power_percent,NF3_Stick_Flow,O3_Stick_Flow,Pressure_Chamber,Pressure_Foreline,Purge03_Stick_pressure,RPS_Status,Step_Number,Temperature_Heater,Throttle_Valve_Position,UVLampGen_rActualPower_Bulb_A,UVLampGen_rActualPower_Bulb_B,UVLampRotator_rCurPosDegrees,UV_Lamp_On_Time,UV_Lamp_Stand_By_On_Time,Step_Name,stpid,CalcStepLoopCnt
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
422801.0,-5.3,-0.2,0.0,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,10032.0,0.016,0.004,NaN,0.0,8.0,300.2,90.0,NaN,NaN,NaN,NaN,NaN,Pump to base,1,1.0
422801.0,-5.3,-0.2,0.0,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,10032.0,0.016,0.004,NaN,0.0,8.0,300.2,90.0,NaN,NaN,NaN,NaN,NaN,Pump to base,1,1.0
422801.0,-5.3,-0.4,0.0,0.0,NaN,0.0,NaN,74.945055,NaN,40.0,2.252,0.075,59.2,0.0,10032.0,0.016,0.004,0.0,0.0,8.0,300.2,90.0,0.0,0.0,0.0,NaN,NaN,Pump to base,1,1.0
422801.0,-5.3,-0.4,0.0,0.0,NaN,0.0,NaN,74.945055,NaN,40.0,2.252,0.075,59.2,0.0,10032.0,0.016,0.004,0.0,0.0,8.0,300.2,90.0,0.0,0.0,0.0,NaN,NaN,Pump to base,1,1.0
422801.0,-5.3,-0.2,0.0,0.0,NaN,0.0,NaN,74.945055,NaN,40.0,2.252,0.075,59.2,0.0,10032.0,0.016,0.004,0.0,0.0,8.0,300.2,90.0,0.0,0.0,0.0,NaN,NaN,Pump to base,1,1.0


In [17]:
test_norm_data.head()

,ARBTM_Stick_Flow,ARCLN_Stick_Flow,ARSH_Stick_Flow,ARTOP_Stick_Flow,Chamber_pm_wafer_count,HX1_Resistivity,HX1_Temperature,HX4_Temperature,Heater_distance_from_shower_head,Heater_lift_position_from_Home,Heater_output_power_percent,O3_Stick_Flow,Pressure_Chamber,Pressure_Foreline,Step_Number,Temperature_Heater,Throttle_Valve_Position,UV_Lamp_On_Time,UV_Lamp_Stand_By_On_Time,stpid,CalcStepLoopCnt
id,,,,,,,,,,,,,,,,,,,,,
422801.0,0.177778,0.875,0.0,0.000181,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.5,0.00002,0.0,0.888889,0.8,1.0,NaN,NaN,0.0,0.0
422801.0,0.177778,0.875,0.0,0.000181,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.5,0.00002,0.0,0.888889,0.8,1.0,NaN,NaN,0.0,0.0
422801.0,0.177778,0.750,0.0,0.000181,NaN,NaN,0.333333,0.5,1.0,0.0,0.2,0.5,0.00002,0.0,0.888889,0.8,1.0,NaN,NaN,0.0,0.0
422801.0,0.177778,0.750,0.0,0.000181,NaN,NaN,0.333333,0.5,1.0,0.0,0.2,0.5,0.00002,0.0,0.888889,0.8,1.0,NaN,NaN,0.0,0.0
422801.0,0.177778,0.875,0.0,0.000181,NaN,NaN,0.333333,0.5,1.0,0.0,0.2,0.5,0.00002,0.0,0.888889,0.8,1.0,NaN,NaN,0.0,0.0


In [0]:
from sklearn.model_selection import train_test_split
from skimage.util.shape import view_as_windows
import numpy as np

In [19]:
def model_creation(train_data, sample_rate):
    sensor_count = len(train_norm_data.columns)
    window = 2 * max(int(200 / sensor_count), int(1 / sample_rate))
    layers = min(int(np.log2(window * len(train_norm_data.columns))) - 1, 9)
    nodes = [int(2 ** layers / 2 ** i) for i in range(min(5, layers))]
    
    X = view_as_windows(train_norm_data.stack(dropna=False).values, sensor_count * window)
    X_train, X_test = train_test_split(X, test_size=0.2)
    
    print('Window size', window)
    print('Layers and nodes: ', layers, " ", nodes)
    print(X)
    print('X length', len(X))
    print('Length of each record in X', len(X[0]))
    return X
    
model_creation(train_data, sample_rate)

Window size 18
Layers and nodes:  7   [128, 64, 32, 16, 8]
[[8.88888889e-02 7.50000000e-01 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [7.50000000e-01 0.00000000e+00 1.80969405e-04 ... 0.00000000e+00
  0.00000000e+00 1.77777778e-01]
 [0.00000000e+00 1.80969405e-04 0.00000000e+00 ... 0.00000000e+00
  1.77777778e-01 1.00000000e+00]
 ...
 [0.00000000e+00 0.00000000e+00 2.66666667e-01 ... 1.00000000e+00
  1.00000000e+00 1.00000000e+00]
 [0.00000000e+00 2.66666667e-01 3.75000000e-01 ... 1.00000000e+00
  1.00000000e+00 2.50000000e-02]
 [2.66666667e-01 3.75000000e-01 0.00000000e+00 ... 1.00000000e+00
  2.50000000e-02 0.00000000e+00]]
X length 5394754
Length of each record in X 378


#Model Structure
- Encoder module: Iterating through the list of hidden layers encodes the data to its minimum possible form
- Decoder module: From this minimum possible form try to get back the original dataset
- While doing so the algorithm is trying to find the important features contributing